<a href="https://colab.research.google.com/github/Anusri-J/Nasa_Neo_Asteroids_Data/blob/main/Nasa_neo_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
API_KEY = "HL3w7lWjwOGPxdPpihaX2gNFhq7EeoVQoIcta3SK"
asteroids_data = []
target = 10000

import requests
from datetime import datetime

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"


response = requests.get(url)
data = response.json()
details= data['near_earth_objects']
while len(asteroids_data) < target:
      for date,ast_details in details.items():
          for ast in ast_details:
            asteroids_data.append(dict(
                id = int(ast.get('id', 0)),
                neo_ref_id = int(ast.get('neo_reference_id', 0)),
                name = ast.get('name', 'Unknown'),
                magnitude = float(ast.get('absolute_magnitude_h', 0.0)),
                dia_min = float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 0.0)),
                dia_max = float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 0.0)),
                pot_haz = bool(ast.get('is_potentially_hazardous_asteroid', False)),
                close_date = datetime.strptime(ast.get('close_approach_date', '2024-01-01'), "%Y-%m-%d").date(),
                velo = float(ast.get('relative_velocity', {}).get('kilometers_per_hour', 0.0)),
                astronomical = float(ast.get('miss_distance', {}).get('astronomical', 0.0)),
                dis_km = float(ast.get('miss_distance', {}).get('kilometers', 0.0)),
                dis_lunar = float(ast.get('miss_distance', {}).get('lunar', 0.0)),
                orb_body = ast.get('orbiting_body', 'Unknown')


            ))

            if len(asteroids_data) == target:
               break
          if len(asteroids_data) == target:
              break
      url = data['links']['next']


asteroids_data

[{'id': 2415949,
  'neo_ref_id': 2415949,
  'name': '415949 (2001 XY10)',
  'magnitude': 19.37,
  'dia_min': 0.3552670883,
  'dia_max': 0.7944013596,
  'pot_haz': False,
  'close_date': datetime.date(2024, 1, 1),
  'velo': 0.0,
  'astronomical': 0.0,
  'dis_km': 0.0,
  'dis_lunar': 0.0,
  'orb_body': 'Unknown'},
 {'id': 3160747,
  'neo_ref_id': 3160747,
  'name': '(2003 SR84)',
  'magnitude': 26.0,
  'dia_min': 0.0167708462,
  'dia_max': 0.0375007522,
  'pot_haz': False,
  'close_date': datetime.date(2024, 1, 1),
  'velo': 0.0,
  'astronomical': 0.0,
  'dis_km': 0.0,
  'dis_lunar': 0.0,
  'orb_body': 'Unknown'},
 {'id': 3309828,
  'neo_ref_id': 3309828,
  'name': '(2005 YQ96)',
  'magnitude': 20.62,
  'dia_min': 0.1997813652,
  'dia_max': 0.4467247133,
  'pot_haz': True,
  'close_date': datetime.date(2024, 1, 1),
  'velo': 0.0,
  'astronomical': 0.0,
  'dis_km': 0.0,
  'dis_lunar': 0.0,
  'orb_body': 'Unknown'},
 {'id': 3457842,
  'neo_ref_id': 3457842,
  'name': '(2009 HC21)',
  'magn

In [ ]:
len(asteroids_data)

10000

In [ ]:
pip install mysql-connector-python

In [ ]:
import mysql.connector
conn = mysql.connector.connect(host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',user='3EBy9M5V1fh5CGr.root',password='B1NWDKr2YpsTNOyT',database='nasa_asteroid_data')
cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    id INT,
    name VARCHAR(255),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(255)
)
""")
conn.commit()

In [ ]:
insert_asteroids = """
INSERT INTO asteroids (
    id,
    name,
    absolute_magnitude_h,
    estimated_diameter_min_km,
    estimated_diameter_max_km,
    is_potentially_hazardous_asteroid
) VALUES (%s, %s, %s, %s, %s, %s)
"""

asteroids_values = [
    (
        int(ast.get('id', 0)),
        ast.get('name', 'Unknown'),
        float(ast.get('absolute_magnitude_h', 0.0)),
        float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 0.0)),
        float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 0.0)),
        bool(ast.get('is_potentially_hazardous_asteroid', False))
    )
    for ast in asteroids_data
]

cursor.executemany(insert_asteroids, asteroids_values)
conn.commit()

In [ ]:
insert_close_approach = """
INSERT INTO close_approach (
    neo_reference_id,
    close_approach_date,
    relative_velocity_kmph,
    astronomical,
    miss_distance_km,
    miss_distance_lunar,
    orbiting_body
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

close_approach_values = []

for ast in asteroids_data:
    for approach in ast.get('close_approach_data', []):
        close_approach_values.append((
            int(ast.get('id', 0)),
            datetime.strptime(approach.get('close_approach_date', '2024-01-01'), "%Y-%m-%d").date(),
            float(approach.get('relative_velocity', {}).get('kilometers_per_hour', 0.0)),
            float(approach.get('miss_distance', {}).get('astronomical', 0.0)),
            float(approach.get('miss_distance', {}).get('kilometers', 0.0)),
            float(approach.get('miss_distance', {}).get('lunar', 0.0)),
            approach.get('orbiting_body', 'Unknown')
        ))

cursor.executemany(insert_close_approach, close_approach_values)
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM asteroids")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted - 7 columns

,id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid
0,123,2024 AB,22.10,0.093000,0.123000,0
1,2415949,415949 (2001 XY10),19.37,0.355267,0.794401,0
2,3160747,(2003 SR84),26.00,0.016771,0.037501,0
3,3309828,(2005 YQ96),20.62,0.199781,0.446725,1
4,3457842,(2009 HC21),22.10,0.101054,0.225964,0
...,...,...,...,...,...,...
147633,3591616,(2011 YP10),0.00,0.000000,0.000000,0
147634,3608936,(2012 SD22),0.00,0.000000,0.000000,0
147635,3795154,(2017 YD8),0.00,0.000000,0.000000,0
147636,3842680,(2019 KK5),0.00,0.000000,0.000000,0


In [ ]:
cursor.execute("SELECT * FROM close_approach")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted - 7 columns

,neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,123,2024-01-05,21450.5,0.005,134000.0,2.1,Earth
1,2415949,2024-01-01,0.0,0.000,0.0,0.0,Unknown
2,3160747,2024-01-01,0.0,0.000,0.0,0.0,Unknown
3,3309828,2024-01-01,0.0,0.000,0.0,0.0,Unknown
4,3457842,2024-01-01,0.0,0.000,0.0,0.0,Unknown
...,...,...,...,...,...,...,...
21243,3591616,2024-01-01,0.0,0.000,0.0,0.0,Unknown
21244,3608936,2024-01-01,0.0,0.000,0.0,0.0,Unknown
21245,3795154,2024-01-01,0.0,0.000,0.0,0.0,Unknown
21246,3842680,2024-01-01,0.0,0.000,0.0,0.0,Unknown


In [ ]:
import mysql.connector
import pandas as pd

# Connect to your database
conn = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    user="3EBy9M5V1fh5CGr.root",
    password="B1NWDKr2YpsTNOyT",
    database="nasa_asteroid_data"
)

# Function to execute SQL and show result
def show_query(query):
    df = pd.read_sql_query(query, conn)
    display(df)

# Example Query
query1 = '''
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
ORDER BY approach_count DESC
'''
show_query(query1)


/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,approach_count
0,3160747,197
1,3457842,197
2,2415949,197
3,3309828,197
4,3553062,196
...,...,...
107,3520662,188
108,54435948,188
109,3843476,188
110,54151575,188


In [ ]:
query2 = '''
SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY avg_velocity DESC
'''
show_query(query2)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,avg_velocity
0,123,21450.5
1,54418649,0.0
2,3608936,0.0
3,3724393,0.0
4,54419021,0.0
...,...,...
107,54244177,0.0
108,54376461,0.0
109,54418438,0.0
110,54373932,0.0


In [ ]:
query3 = '''
SELECT neo_reference_id, MAX(relative_velocity_kmph) AS top_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY top_velocity DESC
LIMIT 10
'''
show_query(query3)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,top_velocity
0,123,21450.5
1,54416724,0.0
2,3789479,0.0
3,3795154,0.0
4,54048870,0.0
5,3645001,0.0
6,3608936,0.0
7,54419601,0.0
8,54419698,0.0
9,2304640,0.0


In [ ]:
query4_simple = '''
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
HAVING approach_count > 3
'''
show_query(query4_simple)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,approach_count
0,54419601,195
1,54417653,195
2,2304640,190
3,3724393,194
4,54339170,191
...,...,...
106,54338714,195
107,54244177,193
108,54373932,190
109,54382904,190


In [ ]:
query5 = '''
SELECT MONTH(close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY month
ORDER BY approach_count DESC
LIMIT 1
'''
show_query(query5)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,month,approach_count
0,1,21248


In [ ]:
query6 = '''
SELECT neo_reference_id, relative_velocity_kmph
FROM close_approach
ORDER BY relative_velocity_kmph DESC
LIMIT 1
'''
show_query(query6)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,relative_velocity_kmph
0,123,21450.5


In [ ]:
query7 = '''
SELECT neo_reference_id, MIN(miss_distance_km) AS closest_approach
FROM close_approach
GROUP BY neo_reference_id
ORDER BY closest_approach ASC
'''
show_query(query7)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,closest_approach
0,3759280,0.0
1,54244177,0.0
2,2304640,0.0
3,54418796,0.0
4,3724393,0.0
...,...,...
107,54418438,0.0
108,54278142,0.0
109,3780672,0.0
110,2199003,0.0


In [ ]:
query8 = '''
SELECT neo_reference_id, close_approach_date, miss_distance_km
FROM close_approach
ORDER BY neo_reference_id, close_approach_date
'''
show_query(query8)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,close_approach_date,miss_distance_km
0,123,2024-01-05,134000.0
1,2199003,2024-01-01,0.0
2,2199003,2024-01-01,0.0
3,2199003,2024-01-01,0.0
4,2199003,2024-01-01,0.0
...,...,...,...
21243,54514905,2024-01-01,0.0
21244,54514905,2024-01-01,0.0
21245,54514905,2024-01-01,0.0
21246,54514905,2024-01-01,0.0


In [ ]:
query9 = '''
SELECT ca.neo_reference_id, ca.close_approach_date, ca.miss_distance_km
FROM close_approach ca
JOIN (
    SELECT neo_reference_id, MIN(miss_distance_km) AS min_distance
    FROM close_approach
    GROUP BY neo_reference_id
) AS sub
ON ca.neo_reference_id = sub.neo_reference_id AND ca.miss_distance_km = sub.min_distance
'''
show_query(query9)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,close_approach_date,miss_distance_km
0,3787600,2024-01-01,0.0
1,3789479,2024-01-01,0.0
2,3837605,2024-01-01,0.0
3,3986746,2024-01-01,0.0
4,54048870,2024-01-01,0.0
...,...,...,...
21243,3591616,2024-01-01,0.0
21244,3608936,2024-01-01,0.0
21245,3795154,2024-01-01,0.0
21246,3842680,2024-01-01,0.0


In [ ]:
query10 = '''
SELECT neo_reference_id, relative_velocity_kmph
FROM close_approach
WHERE relative_velocity_kmph > 50000
'''
show_query(query10)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,relative_velocity_kmph


In [ ]:
query11 = '''
SELECT MONTH(close_approach_date) AS month, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY month
ORDER BY month
'''
show_query(query11)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,month,total_approaches
0,1,21248


In [ ]:
query12 = '''
SELECT neo_reference_id, close_approach_date, miss_distance_km
FROM close_approach
ORDER BY miss_distance_km ASC
LIMIT 1
'''
show_query(query12)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,close_approach_date,miss_distance_km
0,2415949,2024-01-01,0.0


In [ ]:
query13 = '''
SELECT neo_reference_id, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY neo_reference_id
ORDER BY total_approaches DESC
'''
show_query(query13)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,total_approaches
0,3457842,197
1,2415949,197
2,3309828,197
3,3160747,197
4,3842680,196
...,...,...
107,54151575,188
108,3843476,188
109,54418797,188
110,3398095,188


In [ ]:
query14 = '''
SELECT neo_reference_id, close_approach_date, miss_distance_lunar
FROM close_approach
WHERE miss_distance_lunar < 1
'''
show_query(query14)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,close_approach_date,miss_distance_lunar
0,2415949,2024-01-01,0.0
1,3160747,2024-01-01,0.0
2,3309828,2024-01-01,0.0
3,3457842,2024-01-01,0.0
4,3553062,2024-01-01,0.0
...,...,...,...
21242,3591616,2024-01-01,0.0
21243,3608936,2024-01-01,0.0
21244,3795154,2024-01-01,0.0
21245,3842680,2024-01-01,0.0


In [ ]:
query15 = '''
SELECT neo_reference_id, close_approach_date, astronomical AS miss_distance_au
FROM close_approach
WHERE astronomical < 0.05
'''
show_query(query15)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,close_approach_date,miss_distance_au
0,123,2024-01-05,0.005
1,2415949,2024-01-01,0.000
2,3160747,2024-01-01,0.000
3,3309828,2024-01-01,0.000
4,3457842,2024-01-01,0.000
...,...,...,...
21243,3591616,2024-01-01,0.000
21244,3608936,2024-01-01,0.000
21245,3795154,2024-01-01,0.000
21246,3842680,2024-01-01,0.000


In [ ]:
#Asteroid with the highest number of approaches in a single month

query16 = '''
SELECT neo_reference_id, MONTH(close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id, MONTH(close_approach_date)
ORDER BY approach_count DESC
LIMIT 1
'''
show_query(query16)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,month,approach_count
0,3457842,1,197


In [ ]:
#Year with the highest total asteroid approaches

query17 = '''
SELECT YEAR(close_approach_date) AS year, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY year
ORDER BY total_approaches DESC
LIMIT 1
'''
show_query(query17)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,year,total_approaches
0,2024,21248


In [ ]:
#All unique orbiting bodies asteroids have approached

query18 = '''
SELECT DISTINCT orbiting_body
FROM close_approach
'''
show_query(query18)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,orbiting_body
0,Unknown
1,Earth


In [ ]:
#Asteroids with multiple approaches in the same week

query19 = '''
SELECT neo_reference_id, WEEK(close_approach_date) AS week_num, COUNT(*) AS approaches
FROM close_approach
GROUP BY neo_reference_id, WEEK(close_approach_date)
HAVING approaches > 1
'''
show_query(query19)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,week_num,approaches
0,3797695,0,194
1,54377575,0,192
2,2450293,0,193
3,54382904,0,190
4,54415081,0,195
...,...,...,...
106,54212701,0,188
107,54419812,0,190
108,54418438,0,194
109,3724393,0,194


In [ ]:
#Top 5 asteroids with the longest span between first and last approach

query20 = '''
SELECT neo_reference_id,
       MIN(close_approach_date) AS first_approach,
       MAX(close_approach_date) AS last_approach,
       DATEDIFF(MAX(close_approach_date), MIN(close_approach_date)) AS span_days
FROM close_approach
GROUP BY neo_reference_id
ORDER BY span_days DESC
LIMIT 5
'''
show_query(query20)

/tmp/ipython-input-127-3384109146.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,neo_reference_id,first_approach,last_approach,span_days
0,54100190,2024-01-01,2024-01-01,0
1,54418796,2024-01-01,2024-01-01,0
2,3102756,2024-01-01,2024-01-01,0
3,3608936,2024-01-01,2024-01-01,0
4,54048870,2024-01-01,2024-01-01,0


In [ ]:
!pip install streamlit streamlit_option_menu

In [ ]:
%%writefile nasa_project.py

import mysql.connector
import pandas as pd
import streamlit as st

# 🔗 Connect to MySQL database
conn = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    user="3EBy9M5V1fh5CGr.root",
    password="B1NWDKr2YpsTNOyT",
    database="nasa_asteroid_data",
    port=4000
)

# Helper to execute SQL queries
def run_query(query):
    return pd.read_sql_query(query, conn)

# Streamlit UI config
st.set_page_config("NASA Asteroid Dashboard", layout="wide")
st.title("☄️ NASA Asteroid SQL Dashboard")

# Query Dictionary
query_options = {
    "1. Count of asteroid approaches":
        '''SELECT neo_reference_id, COUNT(*) AS approach_count
           FROM close_approach GROUP BY neo_reference_id''',

    "2. Average velocity per asteroid":
        '''SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
           FROM close_approach GROUP BY neo_reference_id''',

    "3. Top 10 fastest asteroids":
        '''SELECT neo_reference_id, MAX(relative_velocity_kmph) AS top_velocity
           FROM close_approach GROUP BY neo_reference_id
           ORDER BY top_velocity DESC LIMIT 10''',

    "4. Asteroids with more than 3 approaches":
        '''SELECT neo_reference_id, COUNT(*) AS approach_count
           FROM close_approach GROUP BY neo_reference_id HAVING approach_count > 3''',

    "5. Month with most asteroid approaches":
        '''SELECT MONTH(close_approach_date) AS month, COUNT(*) AS count
           FROM close_approach GROUP BY month ORDER BY count DESC LIMIT 1''',

    "6. Asteroid with the fastest approach":
        '''SELECT neo_reference_id, relative_velocity_kmph
           FROM close_approach ORDER BY relative_velocity_kmph DESC LIMIT 1''',

    "7. Asteroids ordered by closest approach (km)":
        '''SELECT neo_reference_id, MIN(miss_distance_km) AS closest_approach
           FROM close_approach GROUP BY neo_reference_id ORDER BY closest_approach ASC''',

    "8. Closest approaches over time":
        '''SELECT neo_reference_id, close_approach_date, miss_distance_km
           FROM close_approach ORDER BY neo_reference_id, close_approach_date''',

    "9. Closest approach per asteroid":
        '''SELECT ca.neo_reference_id, ca.close_approach_date, ca.miss_distance_km
           FROM close_approach ca
           JOIN (
               SELECT neo_reference_id, MIN(miss_distance_km) AS min_distance
               FROM close_approach GROUP BY neo_reference_id
           ) AS sub
           ON ca.neo_reference_id = sub.neo_reference_id AND ca.miss_distance_km = sub.min_distance''',

    "10. Asteroids with velocity > 50,000 km/h":
        '''SELECT neo_reference_id, relative_velocity_kmph
           FROM close_approach WHERE relative_velocity_kmph > 50000''',

    "11. Count of approaches per month":
        '''SELECT MONTH(close_approach_date) AS month, COUNT(*) AS total_approaches
           FROM close_approach GROUP BY month ORDER BY month''',

    "12. Closest ever asteroid approach (km)":
        '''SELECT neo_reference_id, close_approach_date, miss_distance_km
           FROM close_approach ORDER BY miss_distance_km ASC LIMIT 1''',

    "13. Asteroids with most frequent approaches":
        '''SELECT neo_reference_id, COUNT(*) AS total_approaches
           FROM close_approach GROUP BY neo_reference_id ORDER BY total_approaches DESC''',

    "14. Asteroids closer than 1 Lunar Distance":
        '''SELECT neo_reference_id, close_approach_date, miss_distance_lunar
           FROM close_approach WHERE miss_distance_lunar < 1''',

    "15. Asteroids that came within 0.05 AU":
        '''SELECT neo_reference_id, close_approach_date, astronomical AS miss_distance_au
           FROM close_approach WHERE astronomical < 0.05''',

    # Additional 5:
    "16. Asteroid with most approaches in a single month":
        '''SELECT neo_reference_id, MONTH(close_approach_date) AS month, COUNT(*) AS approach_count
           FROM close_approach GROUP BY neo_reference_id, MONTH(close_approach_date)
           ORDER BY approach_count DESC LIMIT 1''',

    "17. Year with most total asteroid approaches":
        '''SELECT YEAR(close_approach_date) AS year, COUNT(*) AS total_approaches
           FROM close_approach GROUP BY year ORDER BY total_approaches DESC LIMIT 1''',

    "18. Unique orbiting bodies encountered":
        '''SELECT DISTINCT orbiting_body FROM close_approach''',

    "19. Asteroids with multiple approaches in same week":
        '''SELECT neo_reference_id, WEEK(close_approach_date) AS week_num, COUNT(*) AS approaches
           FROM close_approach GROUP BY neo_reference_id, WEEK(close_approach_date)
           HAVING approaches > 1''',

    "20. Top 5 asteroids with longest timespan between approaches":
        '''SELECT neo_reference_id, MIN(close_approach_date) AS first_approach,
                  MAX(close_approach_date) AS last_approach,
                  DATEDIFF(MAX(close_approach_date), MIN(close_approach_date)) AS span_days
           FROM close_approach
           GROUP BY neo_reference_id
           ORDER BY span_days DESC
           LIMIT 5'''
}

# Sidebar widgets
selected_label = st.sidebar.selectbox("Select a predefined query", list(query_options.keys()))
st.sidebar.header("Custom Filters")

start_date = st.sidebar.date_input("Start Date")
end_date = st.sidebar.date_input("End Date")
au_range = st.sidebar.slider("Astronomical Units (AU)", 0.0, 1.0, (0.0, 0.05))
ld_range = st.sidebar.slider("Lunar Distance (LD)", 0.0, 10.0, (0.0, 1.0))
velocity_min = st.sidebar.slider("Min Velocity (km/h)", 0, 100000, 50000)

# Display selected query output
if selected_label:
    st.subheader(f"Output: {selected_label}")
    try:
        df = run_query(query_options[selected_label])
        st.dataframe(df, use_container_width=True)
    except Exception as e:
        st.error(f"Error executing query: {e}")

# Filter-based dynamic SQL query
st.subheader("Filtered Asteroid Approaches")

custom_query = f'''
SELECT *
FROM close_approach
WHERE close_approach_date BETWEEN '{start_date}' AND '{end_date}'
  AND astronomical BETWEEN {au_range[0]} AND {au_range[1]}
  AND miss_distance_lunar BETWEEN {ld_range[0]} AND {ld_range[1]}
  AND relative_velocity_kmph >= {velocity_min}
'''

try:
    filtered_df = run_query(custom_query)
    st.dataframe(filtered_df, use_container_width=True)
except Exception as e:
    st.error(f"Error in filtered query: {e}")

Overwriting nasa_project.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.243.242.13


In [ ]:
! streamlit run nasa_project.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.243.242.13:8501

⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://petite-dingos-shave.loca.lt
/content/nasa_project.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
  Stopping...
^C
